In [42]:
# Импортнем все сразу, чтобы не вспоминать
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, defaultdict
import cv2
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.models import resnet18
from torchvision.datasets import VOCDetection

from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm, trange

# Object Detection

У нас достаточно инструментов и знаний, чтобы попробовать сделать детекцию объектов.

## Определимся сначала с постановкой задачи

Пусть у нас есть картинки:
![Картинки](./img/plain-img.png)

Поручим людям пройтись по картинкам и отметить все интересующие нас объекты. Это можно сделать например следующими способами:

- boundary box (bbox или bb) -- прямоугольник `[x, y, w, h]` со сторонами параллельными краям картинки (самый простой вариант)
- instance/semantic mask -- картинка с нулями и единицами размером с исходную картинку, показывающая какие пиксели относятся к объекту (самый затратный вариант)

![Разметка](./img/img-with-annotation.png)

Можно размечать иначе под конкретные нужды:
- center + radius `[x, y, r]`
- rotated bbox `[angle, x, h, w, h]`
- ломанная окружающая линия `[(x_0, y_0), ... (x_n, y_n)]`
- whatever


**Итак: мы хотим сделать сеть, которая будет с одной картинки предсказывать положение (bbox) и класс нескольких объектов (из N классов)**


## Интерпретация feature maps

![](./img/backbone.png)

Если мы прогоним картинку `[3, W, H]` через привычную сверточную сеть (возьмем например классификационную сетку до GAP), мы получим тензор с большим количеством каналов небольшого пространственного размера `[ch, w, h]` (пространственные размеры пускай уменьшились K раз, K=32 для resnet50 **TODO: проверить размеры**).
Можно грубо сказать что каждый пиксель выходного тензора отвечает за область `KxK` пикселей входной картинки (рецептивно поле однако накрывает всю картинку с запасом). 

![](./img/img-with-grid.png)

Мы уже пытались интерпретировать пиксели в этом тензоре как Class Activation Map. Показывает что-то интересное, но как детектор явно не получится использовать.
Однако мы можем сформулировать детекцию как оптимизационную задачу и проучить модель специально под нее.
Давайте пойдем от идеи CAM и повесим на каждый пиксель выходной мапы несколько голов, которые будут предсказывать необходимые для детекции вещи. 

Договоримся, что пиксель относится к объекту только если его центр (точнее центр BBox'а) попадает в область действия пикселя.

![](./img/image-ssd.png)

Чтобы завести детекцию нам потребуются такие головы:

- `bbox regression` - регрессия bbox'а
- `objectness` - относится ли пиксель к объекту или нет
- `clf` - если относится, то давайте предскажем класс


Как мы когда-то обсуждали, Conv1x1 действует на каждый пиксель точно так же как FC-слой. Так что для наших нужд нам достаточно добавить к модели Conv1x1 с количеством выходных каналов `4 + 1 + N_classes`.

In [ ]:
def monkey_forward(net, x):
    x = net.conv1(x)
    x = net.bn1(x)
    x = net.relu(x)
    x = net.maxpool(x)

    x = net.layer1(x)
    x = net.layer2(x)
    x = net.layer3(x)
    x = net.layer4(x)
    
    x = net.final(x)
    # x = net.avgpool(x)
    # x = torch.flatten(x, 1)
    # x = net.fc(x)
    return x

# Adapt torchvision classification model for detection

На первый взгляд нам надо только выбросить avgpool, однако [forward метод в torchvision.model.resnet написан неудачно для наших целей](https://pytorch.org/vision/stable/_modules/torchvision/models/resnet.html#resnet18), так что придется его запатчить.

**JFYI: подмена логики в рантайме (ака [monkey patching](https://en.wikipedia.org/wiki/Monkey_patch)) -- распространенная но _опасная_ практика**

# Prediction transformations

**Как превратить предсказанные числа в координаты bbox'ов?**

Пусть один пиксель выходного тензора относится к патчу (kx, ky) входной картинки.
BBox'ы живут в пространстве исходных картинок, непосредственные выходы сети -- в пространстве выходных тензоров.

Для удобства вычислений, давайте представим bbox как координаты центра (так будет удобнее регрессировать) + ширина и высота. Предсказываем 4 числа: $t_x, t_y, t_w, t_h$. `scale` -- это цена выходного пикселя во входных.

$$
x_c = s \cdot (\tanh t_x + i + 0.5)\\
y_c = s \cdot (\tanh t_y + j + 0.5)\\
w = s \cdot \exp{t_w}\\
h = s \cdot \exp{t_h}
$$

**NB: Обычно используют так называемые anchor -- затравочные bbox'ы, см whiteboard**

In [65]:
class VeryModel(nn.Module):
    def __init__(self, n_classes=12, cfg=None):
        super().__init__()
        self.n_classes = n_classes
        self.cfg = cfg
        model = resnet18(pretrained=True)
        # добавим в модельку слой и подменим forward
#         model.final = nn.Conv2d(model.fc.in_features, 4 + 1 + n_classes, 1)
#         model.forward = lambda x: monkey_forward(model, x)
        self.inner = model

    def forward(self, x):
        output = self.inner(x)
        
        return dict(
            bboxes=...,
            conf=....,
            clf=...,
        )
    
    def compute_all(self, batch, device=None):
        pass
    
    
net = VeryModel()
net = net.eval()
with torch.no_grad():
    x = torch.zeros((1, 3, 416, 246))
    out = net(x)
    for k, v in out.items():
        print(k, v.shape)

cx torch.Size([1, 1, 13, 8])
cy torch.Size([1, 1, 13, 8])
ww torch.Size([1, 1, 13, 8])
hh torch.Size([1, 1, 13, 8])
obj torch.Size([1, 1, 13, 8])
cls torch.Size([1, 12, 13, 8])


In [63]:
x.repeat?

# Let's prepare data


In [55]:
CLASSES = [
    'person', 'chair', 'car', 'dog', 'bottle', 'cat', 'bird', 'pottedplant', 
    'sheep', 'boat', 'aeroplane', 'tvmonitor', 'bicycle', 'sofa', 
    'horse', 'motorbike', 'diningtable', 'cow', 'train', 'bus',
]

cls2idx = {k: i for i, k in enumerate(CLASSES)}


def process_image(pil_image):
    img = np.asarray(pil_image)
    img = img.astype(np.float32) / 255.0 # img \in [0, 1]
    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 1, 3)
    std =  np.array([0.229, 0.224, 0.225]).reshape(1, 1, 3)
    img = (img - mean) / std
    img = img.astype(np.float32)
    img = np.transpose(img, [2, 0, 1])
    return img

class Verydet:
    def __init__(self, root, image_set="train", download=True):
        self.dataset = VOCDetection(root, image_set=image_set, download=download)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, item):
        pil_image, ddict = self.dataset[item]
        img = process_image(pil_image)
        img = torch.tensor(img)        
        ddict = ddict['annotation']
        
        objects = []
        bboxes = []
        for x in ddict['object']:
            objects.append(cls2idx[x['name']])
            bb = {k: int(v) for k, v in x['bndbox'].items()}
            bboxes.append(tuple(bb[k] for k in ['xmin', 'ymin', 'xmax', 'ymax']))
        ret = {"img": img, "cls": objects, "bboxes": bboxes}
        return ret


trainset = Verydet("./voc", image_set="train", download=True)

Using downloaded and verified file: ./voc/VOCtrainval_11-May-2012.tar


In [56]:
trainset[0]

{'img': tensor([[[ 2.2489,  2.2489,  2.2489,  ...,  1.3413,  1.3584,  1.3755],
          [ 2.2489,  2.2489,  2.2489,  ...,  1.3584,  1.3584,  1.3413],
          [ 2.2489,  2.2489,  2.2489,  ...,  1.4098,  1.3927,  1.3927],
          ...,
          [ 1.3927,  1.2043,  1.4098,  ...,  0.2282, -0.0629, -0.0801],
          [ 1.1700,  1.1872,  1.1872,  ..., -0.2171, -0.2684, -0.0287],
          [ 0.9303,  0.9646,  1.0502,  ..., -0.7137, -0.8678, -0.7479]],
 
         [[ 2.4286,  2.4286,  2.4286,  ...,  1.5532,  1.5707,  1.5882],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.5707,  1.5707,  1.5532],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.6232,  1.6057,  1.6057],
          ...,
          [ 1.3081,  1.1155,  1.3256,  ...,  0.2052, -0.0749, -0.0574],
          [ 1.0805,  1.0980,  1.0980,  ..., -0.1800, -0.2325,  0.0476],
          [ 0.8354,  0.8704,  0.9580,  ..., -0.6877, -0.8452, -0.6877]],
 
         [[ 2.6400,  2.6400,  2.6400,  ...,  2.5180,  2.5354,  2.5529],
          [ 2.6400,  

# WOK 
_Pascal VOC EDA_

Просто посмотреть, что у нас в датасете лежит

In [52]:
ds = VOCDetection("./voc", image_set="train")

some_stats = defaultdict(list)
N = len(ds)
for i in trange(N):
    pic, ddict = ds[i]
    anno = ddict['annotation']
    w = int(anno['size']['width'])
    h = int(anno['size']['height'])
    d = int(anno['size']['depth'])
    
    some_stats['w'].append(w)
    some_stats['h'].append(h)
    some_stats['d'].append(d)
    
    some_stats['objects_per_image'].append(len(anno['object']))
    for x in anno['object']:
        name = x['name']
        bb = x['bndbox']
        ww = int(bb['xmax']) - int(bb['xmin'])
        hh = int(bb['ymax']) - int(bb['ymin'])
        aspect = (ww / hh + 1e-5)
        rel_ww = ww / w
        rel_hh = hh / h
        
        some_stats['name'].append(name)
        some_stats['ww'].append(ww)
        some_stats['hh'].append(hh)
        some_stats['aspect'].append(aspect)
        some_stats['rel_ww'].append(rel_ww)
        some_stats['rel_hh'].append(rel_hh)

In [54]:
# names
print(sorted(Counter(some_stats['name']).items(), key=lambda t: -t[1]))
del some_stats['name']
for k, v in some_stats.items():
    pass
#     plt.figure()
#     plt.title(k)
#     sns.distplot(v, kde=False)
#     plt.show()

['person', 'chair', 'car', 'dog', 'bottle', 'cat', 'bird', 'pottedplant', 'sheep', 'boat', 'aeroplane', 'tvmonitor', 'bicycle', 'sofa', 'horse', 'motorbike', 'diningtable', 'cow', 'train', 'bus']


In [7]:
class Trainer:
    def __init__(self, model: nn.Module,
                 batch_size: int = 128):
        self.model = model
        self.batch_size = batch_size

        run_folder = Path(os.get_cwd())
        train_log_folder = run_folder / "train_log"
        val_log_folder = run_folder / "val_log"
        print(f"Run output folder is {run_folder}")
        os.makedirs(run_folder)
        os.makedirs(train_log_folder)
        os.makedirs(val_log_folder)

        self.outpath = outpath
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0
        self.train_writer = SummaryWriter(log_dir=str(train_log_folder))
        self.val_writer = SummaryWriter(log_dir=str(val_log_folder))

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)

    def train(self, num_epochs: int):
        model = self.model
        optimizer = model.get_optimizer()

        train_loader = model.get_loader(train=True)
        val_loader = model.get_loader(train=False)

        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                model.post_train_batch()
                for k, v in details.items():
                    self.train_writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1

            model.eval()
            val_losses = []
            for batch in tqdm(val_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())

            val_loss = np.mean(val_losses)
            model.post_val_stage(val_loss)

            if val_loss < best_loss:
                self.save_checkpoint(str(self.run_folder / "best_checkpoint.pth"))
                best_loss = val_loss

    def find_lr(self, min_lr: float = 1e-6,
                max_lr: float = 1e-1,
                num_lrs: int = 20,
                smooth_beta: float = 0.8) -> dict:
        lrs = np.geomspace(start=min_lr, stop=max_lr, num=num_lrs)
        logs = {'lr': [], 'loss': [], 'avg_loss': []}
        avg_loss = None
        model = self.model
        optimizer = model.get_optimizer()
        train_loader = model.get_loader(train=True)

        model.train()
        for lr, batch in tqdm(zip(lrs, train_loader), desc='finding LR', total=num_lrs):
            # apply new lr
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

            # train step
            batch = {k: v.to(self.device) for k, v in batch.items()}
            loss, details = model.compute_all(batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # calculate smoothed loss
            if avg_loss is None:
                avg_loss = loss
            else:
                avg_loss = smooth_beta * avg_loss + (1 - smooth_beta) * loss

            # store values into logs
            logs['lr'].append(lr)
            logs['avg_loss'].append(avg_loss)
            logs['loss'].append(loss)

        logs.update({key: np.array(val) for key, val in logs.items()})

        return logs

In [ ]:
# Запустите тренировку, подберите LR
# Добавьте вывод bbox'ов